In [26]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets

In [27]:
BATCH_SIZE = 64
lr                 = 0.001
EPOCHS       = 40
USE_CUDA   = torch.cuda.is_available()
DEVICE        = torch.device("cuda" if USE_CUDA else "cpu")

In [28]:
# sequential=True: 순차적인 데이터셋인가?
# batch_first=True: 신경망에 입력되는 텐서의 첫번째 차원값이 batch_size가 되도록.
# lower=True : 모든 영문 알파벳이 소문자가 되도록.
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [29]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [30]:
TEXT.build_vocab(trainset, min_freq=5)    # 최소 5번 이상 등장한 단어만을 사전에 담겠다. 5번 미만 출현단어는 unk 토큰으로 대체.
LABEL.build_vocab(testset)

In [31]:
trainset, valset = trainset.split(split_ratio=0.8)
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (trainset, valset, testset),
    batch_size=BATCH_SIZE,
    shuffle=True,
    repeat=False
)

In [32]:
vocab_size = len(TEXT.vocab)
n_classes = 2

In [33]:
print("[학습셋]: %d [검증셋]: %d [테스트셋]: %d [단어 수]: %d [클래스]: %d" 
      % (len(trainset), len(valset), len(testset), vocab_size, n_classes))

[학습셋]: 20000 [검증셋]: 5000 [테스트셋]: 25000 [단어 수]: 46159 [클래스]: 2


### RNN 모델 구현

In [34]:
class BasicGRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, dropout_p=0.2):
        super(BasicGRU, self).__init__()
        print("Building Basic GRU model...")
        self.n_layers = n_layers
        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(
            embed_dim,
            self.hidden_dim,
            num_layers=self.n_layers,
            batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)
    
    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0))
        x, _ = self.gru(x, h_0)
        h_t = x[:, -1, :]
        self.dropout(h_t)
        logit = self.out(h_t)
        return logit
    
    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [35]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)    # label값을 0, 1 로 변환
        optimizer.zero_grad()
        
        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()                      

In [37]:
def evaluate(model, val_iter):
    """ Evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy        

In [38]:
model = BasicGRU(
    n_layers=1,
    hidden_dim=256,
    n_vocab=vocab_size,
    embed_dim=128,
    n_classes=n_classes,
    dropout_p=0.5).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Building Basic GRU model...


In [39]:
best_val_loss  = None
for e in range(1, EPOCHS+1):
        train(model, optimizer, train_iter)
        val_loss, val_accuracy = evaluate(model, val_iter)
        
        print("[Epoch: %d] 검증 오차: %5.2f | 검증 정확도: %5.2f" %
              (e, val_loss, val_accuracy))
        if not best_val_loss or val_loss < best_val_loss:
            if not os.path.isdir("snapshot"):
                os.makedirs("snapshot")
            torch.save(model.state_dict(), './snapshot/txtclassification.pt')
            best_val_loss = val_loss

[Epoch: 1] 검증 오차:  0.69 | 검증 정확도: 51.10
[Epoch: 2] 검증 오차:  0.70 | 검증 정확도: 49.64
[Epoch: 3] 검증 오차:  0.59 | 검증 정확도: 70.36
[Epoch: 4] 검증 오차:  0.57 | 검증 정확도: 68.24
[Epoch: 5] 검증 오차:  0.40 | 검증 정확도: 83.48
[Epoch: 6] 검증 오차:  0.39 | 검증 정확도: 82.82
[Epoch: 7] 검증 오차:  0.40 | 검증 정확도: 82.92
[Epoch: 8] 검증 오차:  0.35 | 검증 정확도: 84.60
[Epoch: 9] 검증 오차:  0.36 | 검증 정확도: 84.78
[Epoch: 10] 검증 오차:  0.37 | 검증 정확도: 84.94
[Epoch: 11] 검증 오차:  0.36 | 검증 정확도: 85.46
[Epoch: 12] 검증 오차:  0.42 | 검증 정확도: 84.64
[Epoch: 13] 검증 오차:  0.38 | 검증 정확도: 85.80
[Epoch: 14] 검증 오차:  0.45 | 검증 정확도: 84.32
[Epoch: 15] 검증 오차:  0.43 | 검증 정확도: 84.62
[Epoch: 16] 검증 오차:  0.46 | 검증 정확도: 84.32
[Epoch: 17] 검증 오차:  0.44 | 검증 정확도: 84.56
[Epoch: 18] 검증 오차:  0.46 | 검증 정확도: 84.40
[Epoch: 19] 검증 오차:  0.48 | 검증 정확도: 84.86
[Epoch: 20] 검증 오차:  0.55 | 검증 정확도: 83.08
[Epoch: 21] 검증 오차:  0.60 | 검증 정확도: 81.68
[Epoch: 22] 검증 오차:  0.49 | 검증 정확도: 84.48
[Epoch: 23] 검증 오차:  0.53 | 검증 정확도: 83.94
[Epoch: 24] 검증 오차:  0.47 | 검증 정확도: 86.74
[Epoch: 25] 검증 오차:  0.53 

In [40]:
model.load_state_dict(torch.load('./snapshot/txtclassification.pt'))
test_loss, test_acc = evaluate(model, test_iter)
print("테스트 오차: %5.2f | 테스트 정확도: %5.2f" % (test_loss, test_acc))

테스트 오차:  0.37 | 테스트 정확도: 83.66
